In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os, glob
import gc

import time
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cv2
from skimage import io
from skimage.color import rgb2gray

from fast_slic.avx2 import SlicAvx2
from skimage.segmentation import mark_boundaries

import imgaug.augmenters as iaa

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

from sklearn.metrics import accuracy_score

In [ ]:
LABELS = ['Branching', 'Fish', 'Massive', 'Not Massive', 'Substrate', 'Target', 'Water']

labels = {'Branching' : 0, 
          'Fish' : 1, 
          'Massive' : 2,
          'Not Massive' : 3,
          'Substrate' : 4,
          'Target' : 5,
          'Water' : 6}

NO_LABEL = 255
N_CLASSES = 7

import seaborn as sns
cp = sns.color_palette("hls", 7)

In [ ]:
def colorize_prediction(pred):
   
    colored_mask = np.zeros(shape = (pred.shape[0], pred.shape[1], 3))

    for _ in np.unique(pred):
           
            colored_mask[pred == _] = cp[_]
        
    return colored_mask

def display(gt, mask):

    plt.figure(figsize = (20, 15))
    
    plt.subplot(1, 2, 1)
    plt.imshow(gt, alpha = .65)

    plt.subplot(1, 2, 2)
    plt.imshow(mask, alpha = .65)
    plt.show()
    
    
def compute_segmentations(iterations, start, end):
    # change how you calculate the number of segments each iteration
    # my method was linear, Shathe's is exponential (much better)
    num_iters = iterations
    start_sp = start
    end_sp = end

    reduction_factor = math.pow(float(end_sp) / start_sp, 1. / (num_iters - 1))

    return [int(start_sp * math.pow(reduction_factor, iters)) for iters in np.arange(num_iters)]


# SciPy's Mode function with an addition to choose the 2nd most common
# value in a column (through the 3rd dimension) if the 1st most common
# value is 225
def mode(a, axis = 0):

    a = np.array(a)
    scores = np.unique(np.ravel(a))  # all unique values
    testshape = list(a.shape)        # dimensions of array
    testshape[axis] = 1
    oldmostfreq = np.zeros(testshape, dtype = int)    
    oldcounts = np.zeros(testshape, dtype = int)       

    for score in scores:

        if(score == NO_LABEL):
            continue

        template = (a == score)                                 
        counts = np.expand_dims(np.sum(template, axis), axis)

        mostfrequent = np.where(counts > oldcounts, score, oldmostfreq)  
        oldcounts = np.maximum(counts, oldcounts)
        oldmostfreq = mostfrequent

    return mostfrequent[0]

In [ ]:
def get_sparse_points(file, threshold):
        
    sparse_points = pd.read_csv(file)   

    sparse_points = sparse_points[sparse_points['Confidence'] >= threshold]
    
    sparse_points = sparse_points.drop('Unnamed: 0', axis = 1)
    sparse_points = sparse_points.reset_index(drop = True)

    return sparse_points

In [ ]:
# the algorithm that uses SLIC for multiple iterations
def get_mask(image, annotation, iterations, start, end, method):
    
    all_masks = []
    segmentations = compute_segmentations(iterations, start, end)

    # Loops through, each time a mask is created with different settings, and is accumulated
    for _ in range(iterations):
        
        n_segments = segmentations[_]

        # Uses CPU to create segmented image with current params
        slic = SlicAvx2(num_components = n_segments, compactness = 20)
        segmented_image = slic.iterate(cv2.cvtColor(image, cv2.COLOR_RGB2LAB))

        # The XY location of each annotation, along with the label
        X = np.array(annotation['X'].values, dtype = 'uint16')
        Y = np.array(annotation['Y'].values, dtype = 'uint16')
        L = np.array(annotation['Labels'].values, dtype = 'str')  
    
        # By plugging in the annotation locations into the segmented image
        # you get all of the segment IDs that contain annotations (desired segments, DS)
        # Then for each DS, find the class label for the annotations within it (color label, CL)
        DS = segmented_image[Y, X]                               
        CL = np.array([labels.get(L[i]) for i in range(len(DS))])
        
        # If multiple annotations are within the same segment, find
        # the most common label among those annotations and provide it
        # as the final label for that segment (majority rule)
        if(len(DS) != len(np.unique(DS))):

            for segment_ID in np.unique(DS):
                annotations_in_segment = list(np.where(DS == segment_ID)[0])
                labels_of_annotations = [CL[a] for a in annotations_in_segment]
                most_common_label_in = max(set(labels_of_annotations), key = labels_of_annotations.count)
                CL[annotations_in_segment] = most_common_label_in
        
        # Lastly, combine them into a dictionary, which speeds up the the next process by 50%
        pairs = dict(zip(DS, CL))

        # one mask to hold the labels for each super-pixel (and individual pixels)
        mask = np.full(shape = image.shape[0:2], fill_value = NO_LABEL)

        for index, segVal in enumerate(list(pairs)):
            mask[segmented_image == segVal] = pairs[list(pairs)[index]]
            # provides each individual pixel with the class label of the super-pixel

        # Collects all masks made of this image for all iterations
        all_masks.append(mask)
        
        gc.collect()
                    
    # Now that the loop is over, we have a bunch of segmented images where there are
    # pixels with class labels, and pixels with no label (255). We can combine them
    # following Alonso et al. 2019's join method, or Jordan's mode method
    
    # Starting with the first mask which is the most detailed, we mask it with less a
    # detailed masks that contains more labeled pixels, mask_b. With each iteration, 
    # the class mask (final_mask) fills up with the additional pixels provided by mask_b; 
    # no pixels get replaced by mask_b, they only add to final_mask
    if(method == 'join'):
        # First mask is most detailed
        final_mask = all_masks[0]

        # Go through the remaining masks
        for _ in all_masks[1:]:
            mask_b = _

            # find the pixels in mask_a that are not labelled, assign them with the labels
            # of the pixels in mask_b
            final_mask[final_mask == NO_LABEL] = mask_b[final_mask == NO_LABEL] 
            
    # Jordan's method, 'mode'
    else:
         final_mask = mode(all_masks)
            
    return final_mask

In [ ]:
def get_rates(matrix):

    TP = np.diag(matrix);

    FP = np.sum(matrix, axis = 0) - TP 

    FN = np.sum(matrix, axis = 1) - TP 

    TN = []
    for _ in range(len(np.diag(matrix))):
        temp = np.delete(matrix, _, 0)    
        temp = np.delete(temp, _, 1)  
        TN.append(sum(sum(temp)))

    return TN, FP, FN, TP

def get_scores(ground_truths, predictions):
    
    flat_ground_truth = []
    flat_prediction = []

    # Counters variables
    correct = np.zeros(N_CLASSES)
    actual = np.zeros(N_CLASSES) 
    predicted = np.zeros(N_CLASSES)
    matrix = np.zeros((N_CLASSES, N_CLASSES), np.uint32)

    for _ in range(len(predictions)):
        
        prediction = predictions[_]
        ground_truth = ground_truths[_]
    
        #flat_prediction += prediction.flatten().astype(int).tolist()
        ##flat_ground_truth += ground_truth.flatten().astype(int).tolist()

        # Computes predicted, correct, real and the confusion matrix per file, accumulates all
        for c in range(N_CLASSES):

            # Number of predictions per class
            predicted[c] = predicted[c] + sum(sum((ground_truth >= 0) & (ground_truth < N_CLASSES) & (prediction == c)))

            # Number of correctly predicted samples per class
            correct[c] = correct[c] + sum(sum((ground_truth == c ) & (prediction == c)) )   

            # Number of real samples per class
            actual[c] = actual[c] + sum(sum(ground_truth == c))                   

            # Build a confusion matrix
            for x in range(N_CLASSES):
                matrix[c, x] = matrix[c, x] + sum(sum((ground_truth == c) & (prediction == x)))

    # Calculating metrics    
    TN, FP, FN, TP = get_rates(matrix)

    matrix_normalized = np.around( (matrix / matrix.astype(np.float).sum(axis = 1, keepdims = True)) , 2 )


    print("Relative Abundance: ")

    RA= pd.DataFrame(list(zip(LABELS, np.around(actual/np.sum(actual), 4), np.around(predicted/np.sum(predicted), 4))),
                columns= ['Class Labels', 'Ground-Truth', 'Predictions'])
    print(RA)

    print("\nConfusion Matrix:")
    print(matrix_normalized)

    # Calculates metrics per class (macro), and weights
    p = (TP / (TP + FP))
    r = (TP / (TP + FN))
    dice = (2 * TP) / (TP + FP + FN + TP)
    iou = (TP) / (TP + FP + FN)
    w = actual/np.sum(actual)

    # Overall accuracy using scikit
    print("\nOverall Accuracy: ", accuracy_score(flat_ground_truth, flat_prediction))

    # Per class and Average Class Accuracy
    per_class_accuracy = []
    for _ in range(N_CLASSES):
        per_class_accuracy.append(correct[_]/actual[_])

    print("\nAverage Class Accuracy: ", np.around(np.mean(per_class_accuracy), 4))

    print("\nPer Class Accuracy: ")
    [print(LABELS[_], ":", round(per_class_accuracy[_], 3)) for _ in range(N_CLASSES)]

    # Macro and weighted metrics
    print("\nMacro: ")
    print("Precision:", np.around(np.mean(p), 4))
    print("Recall:", np.around(np.mean(r), 4))
    print("Dice:", np.around(np.mean(dice), 4))
    print("Iou:", np.around(np.mean(iou), 4))

    print("\nWeighted: ")
    print("Precision:", np.around(np.sum([p[_] * w[_] for _ in range(N_CLASSES)]), 4))
    print("Recall:", np.around(np.sum([r[_] * w[_] for _ in range(N_CLASSES)]), 4))
    print("Dice:", np.around(np.sum([dice[_] * w[_] for _ in range(N_CLASSES)]), 4))
    print("IoU:", np.around(np.sum([iou[_] * w[_] for _ in range(N_CLASSES)]), 4))

In [ ]:
benchmarking = True

if(benchmarking):
    path = "ground_truth\\"
else:
    path = "Z:\\Photogrammetry_data\\Florida\\7_16_2019_patch\\models\\before-algae\\Deep Learning\\data\\"


images = sorted(glob.glob(path + "images\\*.png"))
sparse = sorted(glob.glob("Sparse\\Manual 100\\*.csv"))
gts = sorted(glob.glob(path + "numpy\\*.npy"))

data = pd.DataFrame(data = list(zip(images, gts, sparse)), columns = ['images', 'gts', 'sparse'])
data.shape

In [ ]:
ground_truths = []
predictions = []

# Reduction factor: Alonso and I both found that even if you reduce the size of the input image
# the results of the dense labels will quantiatiavely be about the same, but Fast-MSS runs even faster
# cavet is that by using nearest-neighbor interpolation to upsample the dense labels, it becomes more pixelated
rf = 6

h, w = 2160//rf, 3840//rf

iterations, start, end = 30, 5000, 300

for index in range(50):

    print(index)

    name = data['images'][index].split("\\")[-1].split(".")[0]
    ground_truth = np.argmax(np.load(data['gts'][index]), axis = 2)
    image = iaa.Resize({'height' : h, 'width' : w}).augment_image(io.imread(data['images'][index]))

    sparse_points = get_sparse_points(data['sparse'][index], threshold = .75)
    sparse_points['X'] = sparse_points['X'].values//rf
    sparse_points['Y'] = sparse_points['Y'].values//rf

    start_time = time.time()
    mask = get_mask(image, sparse_points, iterations = iterations, start = start, end = end, method = 'mode')
    end_time = time.time() - start_time

    prediction = iaa.Resize({'height' : 2160, 'width' : 3840}, interpolation = 'nearest').augment_image(mask)
    #io.imsave(fname = path + "dense\\" + name + ".png", arr = mask)

    predictions.append(prediction)
    ground_truths.append(ground_truth)

print("Time:", end_time)
get_scores(ground_truths, predictions)

print("\n")   